In [20]:
import pandas as pd
import numpy as np
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

### JSON get

In [2]:
# response = requests.get('http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=01749ef9ac104e669c9133109222811&q=Jakarta&format=json&date=2022-11-22&enddate=2022-11-28&tp=1')
# json_dict = response.json()

In [3]:
# df = pd.DataFrame(json_dict['data']['weather'][0]['hourly'])
# df['date'] = json_dict['data']['weather'][0]['date']

# for item in range(1,7):
#     df['weatherValue'] = df['weatherDesc'].apply(lambda x: x[0]['value'])
#     temp = pd.DataFrame(json_dict['data']['weather'][item]['hourly'])
#     temp['date'] = json_dict['data']['weather'][item]['date']
#     df = pd.concat([df, temp], ignore_index=True)
    

### Read Data

In [6]:
df_traffic_26 = pd.read_csv('datasets/tomtom_2022-11-26.csv')
df_traffic_28 = pd.read_csv('datasets/tomtom_2022-11-28.csv')
df_weather = pd.read_csv('datasets/test2022-11-27.csv')

df_traffic_merge = pd.concat([df_traffic_26, df_traffic_28], ignore_index=True)

### Data Normalization

- Normalize date_time format to hh:00:00
- Add `traffic_category` column to categorize `traffic_index` quantitative attribute

In [7]:
df_traffic_merge = pd.concat([df_traffic_26, df_traffic_28], ignore_index=True)
df_traffic_merge['date_time'] = df_traffic_merge['date_time'].apply(lambda x: x.split(' ')[0] + " " + x.split(' ')[1][:3] + "00:00")
df_traffic_merge['traffic_category'] = df_traffic_merge['traffic_index'].apply(lambda t: "Empty" if t<25 else "Fluid" if t<50 else "Heavy" if t<75 else "Jam")

In [8]:
df_weather.isnull().sum()

date_time       0
weatherValue    0
dtype: int64

In [9]:
df_traffic_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   timestamp              48 non-null     int64  
 1   date_time              48 non-null     object 
 2   traffic_index          48 non-null     int64  
 3   jams_count             48 non-null     int64  
 4   jams_length            48 non-null     float64
 5   jams_delay             48 non-null     float64
 6   traffic_index_weekago  48 non-null     int64  
 7   weekday                48 non-null     object 
 8   traffic_category       48 non-null     object 
dtypes: float64(2), int64(4), object(3)
memory usage: 3.5+ KB


In [10]:
df_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date_time     56 non-null     object
 1   weatherValue  56 non-null     object
dtypes: object(2)
memory usage: 1.0+ KB


In [11]:
df_weather.head()

,date_time,weatherValue
0,2022-11-22 00:00:00,Clear
1,2022-11-22 03:00:00,Clear
2,2022-11-22 06:00:00,Sunny
3,2022-11-22 09:00:00,Sunny
4,2022-11-22 12:00:00,Sunny


In [12]:
df_traffic_merge.head()

,timestamp,date_time,traffic_index,jams_count,jams_length,jams_delay,traffic_index_weekago,weekday,traffic_category
0,1669395690,2022-11-26 00:00:00,2,21,24.7,96.5,2,Saturday,Empty
1,1669399290,2022-11-26 01:00:00,0,15,19.4,68.5,0,Saturday,Empty
2,1669402890,2022-11-26 02:00:00,0,13,20.0,35.0,0,Saturday,Empty
3,1669406490,2022-11-26 03:00:00,0,13,23.4,37.3,0,Saturday,Empty
4,1669410090,2022-11-26 04:00:00,0,13,16.3,24.0,0,Saturday,Empty


### Data Merging

In [13]:
df_merge = pd.merge(df_traffic_merge, df_weather, left_on='date_time', right_on='date_time')
df_merge

,timestamp,date_time,traffic_index,jams_count,jams_length,jams_delay,traffic_index_weekago,weekday,traffic_category,weatherValue
0,1669395690,2022-11-26 00:00:00,2,21,24.7,96.5,2,Saturday,Empty,Patchy rain possible
1,1669406490,2022-11-26 03:00:00,0,13,23.4,37.3,0,Saturday,Empty,Partly cloudy
2,1669417290,2022-11-26 06:00:00,0,12,7.2,42.2,0,Saturday,Empty,Partly cloudy
3,1669430790,2022-11-26 09:00:00,23,199,146.0,826.2,20,Saturday,Empty,Partly cloudy
4,1669441590,2022-11-26 12:00:00,56,590,404.2,2949.6,42,Saturday,Heavy,Overcast
5,1669450590,2022-11-26 15:00:00,81,818,656.7,4241.0,53,Saturday,Jam,Partly cloudy
6,1669460490,2022-11-26 18:00:00,71,587,489.8,3085.0,47,Saturday,Heavy,Partly cloudy
7,1669471290,2022-11-26 21:00:00,30,169,88.3,767.5,31,Saturday,Fluid,Partly cloudy
8,1669568490,2022-11-28 00:00:00,0,9,5.1,31.7,0,Monday,Empty,Patchy rain possible
9,1669579290,2022-11-28 03:00:00,0,3,3.3,2.9,0,Monday,Empty,Light rain shower


In [14]:
df_filter = pd.pivot_table(df_merge, values=['traffic_category'], index=['weatherValue'], columns=['date_time'], aggfunc=np.sum)
df_filter

traffic_category                      \
date_time            2022-11-26 00:00:00 2022-11-26 03:00:00   
weatherValue                                                   
Light rain shower                    NaN                 NaN   
Overcast                             NaN                 NaN   
Partly cloudy                        NaN               Empty   
Patchy rain possible               Empty                 NaN   

                                                              \
date_time            2022-11-26 06:00:00 2022-11-26 09:00:00   
weatherValue                                                   
Light rain shower                    NaN                 NaN   
Overcast                             NaN                 NaN   
Partly cloudy                      Empty               Empty   
Patchy rain possible                 NaN                 NaN   

                                                              \
date_time            2022-11-26 12:00:00 2022-11-26 15:00:00   
weatherValue                                                   
Light rain shower                    NaN                 NaN   
Overcast                           Heavy                 NaN   
Partly cloudy                        NaN                 Jam   
Patchy rain possible                 NaN                 NaN   

                                                              \
date_time            2022-11-26 18:00:00 2022-11-26 21:00:00   
weatherValue                                                   
Light rain shower                    NaN                 NaN   
Overcast                             NaN                 NaN   
Partly cloudy                      Heavy               Fluid   
Patchy rain possible                 NaN                 NaN   

                                                              \
date_time            2022-11-28 00:00:00 2022-11-28 03:00:00   
weatherValue                                                   
Light rain shower                    NaN               Empty   
Overcast                             NaN                 NaN   
Partly cloudy                        NaN                 NaN   
Patchy rain possible               Empty                 NaN   

                                                              \
date_time            2022-11-28 06:00:00 2022-11-28 09:00:00   
weatherValue                                                   
Light rain shower                  Fluid               Heavy   
Overcast                             NaN                 NaN   
Partly cloudy                        NaN                 NaN   
Patchy rain possible                 NaN                 NaN   

                                                              \
date_time            2022-11-28 12:00:00 2022-11-28 15:00:00   
weatherValue                                                   
Light rain shower                    NaN                 NaN   
Overcast                             NaN                 NaN   
Partly cloudy                        NaN                 NaN   
Patchy rain possible               Fluid               Fluid   

                                                              
date_time            2022-11-28 18:00:00 2022-11-28 21:00:00  
weatherValue                                                  
Light rain shower                    Jam                 NaN  
Overcast                             NaN                 NaN  
Partly cloudy                        NaN                 NaN  
Patchy rain possible                 NaN               Fluid

In [15]:
itemsetList = []
dflist = df_filter.values.tolist()
dfindex = df_filter.index.tolist()

for i in range (len(df_filter)):
    currentList = [dflist[i][item] for item in range(len(dflist[i])) if not(pd.isnull(dflist[i][item])) == True]
    itemsetList.append([currentList[item] for item in range(len(currentList)) if currentList[item] != currentList[item-1]] if len(currentList) > 1 else currentList)
    
itemsetList

[['Empty', 'Fluid', 'Heavy', 'Jam'],
 ['Heavy'],
 ['Empty', 'Jam', 'Heavy', 'Fluid'],
 ['Empty', 'Fluid']]

In [16]:
te = TransactionEncoder()
te_ary = te.fit(itemsetList).transform(itemsetList)

In [17]:
df_try = pd.DataFrame(te_ary, columns=te.columns_)
df_try

,Empty,Fluid,Heavy,Jam
0,True,True,True,True
1,False,False,True,False
2,True,True,True,True
3,True,True,False,False


In [18]:
frequent_itemsets = apriori(df_try, min_support=0.6, use_colnames=True)
frequent_itemsets.set_index(pd.Index(dfindex[:len(frequent_itemsets)]), inplace=True)

frequent_itemsets

,support,itemsets
Light rain shower,0.75,(Empty)
Overcast,0.75,(Fluid)
Partly cloudy,0.75,(Heavy)
Patchy rain possible,0.75,"(Empty, Fluid)"


In [19]:
from mlxtend.frequent_patterns import association_rules
res = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)
res.set_index(pd.Index(dfindex[:len(res)]), inplace=True)

res

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
Light rain shower,(Empty),(Fluid),0.75,0.75,0.75,1.0,1.333333,0.1875,inf
Overcast,(Fluid),(Empty),0.75,0.75,0.75,1.0,1.333333,0.1875,inf
